Objetivo

Realizar un análisis sobre la oferta/vidriera de las opciones de productos que responden a distintas búsquedas en el sitio Mercadolibre.com.ar


##**Instalamos e importamos Libreria para trabajar con Apis**

In [69]:
%pip install requests

In [70]:
import requests
import pandas as pd
import time


##**Funciones para obtener informacion de las APIs y crear tablas**

In [71]:
def get_data(base_url, endpoint, params=None, headers=None, expected_field=None):
    """
    Realiza una solicitud GET a una API para obtener datos.

    Parámetros:
    - base_url (str): La URL base de la API.
    - endpoint (str): El endpoint específico de la API.
    - params (dict, opcional): Parámetros de consulta.
    - headers (dict, opcional): Encabezados de la solicitud.
    - expected_field (str, opcional): Campo del JSON donde se espera la data principal (ej: 'results').

    Retorna:
    - dict o list: Datos obtenidos de la API en formato JSON.
    """

    try:
        endpoint_url = f"{base_url}/{endpoint}"
        response = requests.get(endpoint_url, params=params, headers=headers)
        response.raise_for_status()  # Lanza error si la respuesta HTTP es incorrecta.

        # Convierte la respuesta a JSON
        data = response.json()

        # Extrae campo del JSON, extraerlo
        if expected_field and expected_field in data:
            return data[expected_field]

        return data  # Retornar el JSON completo si no se especificó un campo esperado.

    except requests.exceptions.RequestException as e:
        print(f"La petición ha fallado. Código de error : {e}")
        return None

def build_table(json_data):
    """
    Construye un DataFrame de pandas a partir de datos en formato JSON.

    Parámetros:
    json_data (dict): Los datos en formato JSON obtenidos de una API.

    Retorna:
    DataFrame: Un DataFrame de pandas que contiene los datos.
    """
    try:
        df = pd.json_normalize(json_data)
        return df
    except:
        print("Los datos no están en el formato esperado")
        return None


## **Extraccion y almacenamiento de la informacion solicitada**

Uso la api proporcionada con el modo de ejemplo para setear los parametros que me interese modificar.
La inteción es obtener informacion relevante sobre tres items de la misma categoria para poder obtener insights relevantes.
Se deberá modificar el "param_item" segun el producto que queramos obtener la informacion.

###Verifico informacion del ejemplo

In [73]:
#Armo la url del endpoint del BCRA **"https://api.mercadolibre.com/sites/{site}/search?q={item}&limit=50"**

base_url = "https://api.mercadolibre.com"

param_item = "Apple TV" # parametro para editar el producto que quiero analizar probamos con "Amazon Fire TV", "Apple TV", "chromecast"

params = {
    "q": param_item,
    "limit": 50
  }

site = "MLA" # posibilita cambio de site

endpoint_meli = "/sites/" + str(site) + "/search" # recreo el ejemplo para ver la informacion de la API

json_data = get_data(base_url, endpoint_meli, params=params, expected_field="results") # llamo a funcion para traer los datos de la API en formato JSON

# funciones utilizadas para explorar los datos
"""
df_meli = build_table(json_data) # Utilizo esta funcion para pasar el Json a formato de tabla que me facilite la lectura
df_meli.head(5) #veo como es la estructura

valores_columnas = df_meli.columns.values
valores_lista_columnas = list(valores_columnas)
print(valores_lista_columnas) # Obtengo un listado de los atributos del Json
"""

['id', 'title', 'condition', 'thumbnail_id', 'catalog_product_id', 'listing_type_id', 'sanitized_title', 'permalink', 'buying_mode', 'site_id', 'category_id', 'domain_id', 'thumbnail', 'currency_id', 'order_backend', 'price', 'original_price', 'available_quantity', 'official_store_id', 'official_store_name', 'use_thumbnail_id', 'accepts_mercadopago', 'stop_time', 'attributes', 'winner_item_id', 'catalog_listing', 'discounts', 'promotion_decorations', 'promotions', 'inventory_id', 'sale_price.price_id', 'sale_price.amount', 'sale_price.conditions.eligible', 'sale_price.conditions.context_restrictions', 'sale_price.conditions.start_time', 'sale_price.conditions.end_time', 'sale_price.currency_id', 'sale_price.exchange_rate', 'sale_price.payment_method_prices', 'sale_price.payment_method_type', 'sale_price.regular_amount', 'sale_price.type', 'sale_price.metadata.promotion_id', 'sale_price.metadata.promotion_type', 'shipping.store_pick_up', 'shipping.free_shipping', 'shipping.logistic_type

'\n#df_meli.head(5) #veo como es la estructura\n\nvalores_columnas = df_meli.columns.values\nvalores_lista_columnas = list(valores_columnas)\nprint(valores_lista_columnas) # Obtengo un listado de los atributos del Json\n'

Prueba para obtener los Id de los item

In [74]:
# Ver los primeros IDs de productos
item_ids = [item["id"] for item in json_data]
print(item_ids[:10])  # Mostrar los primeros 10

['MLA1473995491', 'MLA1449403463', 'MLA1366426083', 'MLA1555284784', 'MLA1399215546', 'MLA1719499042', 'MLA1981233514', 'MLA1453365954', 'MLA1618134842', 'MLA1594852796']


### Barrido de todos los item_id

In [75]:
all_item_ids = [] # Lista para guardar los item ids
for offset in range(0, 200, 50):  # 4 llamadas: 0, 50, 100, 150
    url = f"https://api.mercadolibre.com/sites/MLA/search?q={param_item}&limit=50&offset={offset}"
    response = requests.get(url)
    data = response.json()
    all_item_ids.extend([item["id"] for item in json_data])

print(len(all_item_ids))  # Debería ser más de 150 y 200 como maximo

200


### Obtengo la informacion de cada Item

Prueba para obtenerel detalle de los items

In [76]:
base_url_items = "https://api.mercadolibre.com/items"

for item_id in item_ids[:5]:  # Solo tomamos los primeros 5 para probar
    item_data = get_data(base_url_items, item_id)
    if item_data:
        print(f"Producto: {item_data['title']}, Precio: {item_data['price']}")
    else:
        print(f"Error al obtener el item {item_id}")

Producto: Apple Tv 4k (wifi + Ethernet) A2843 De Voz 3.ª Generación 2022 4k 128gb Negro - Distribuidor Autorizado, Precio: 499999
Producto: Apple Tv Hd A1625 De Voz 4.ª Generación 2015 Full Hd 32gb Negro Con 2gb De Memoria Ram, Precio: 599990
Producto: Apple Tv 4k (wifi) A2737 De Voz 3.ª Generación 2022 4k 64gb Negro - Distribuidor Autorizado, Precio: 405399
Producto: Apple Tv 4k (wifi + Ethernet) A2843 De Voz 3.ª Generación 2022 4k 128gb Negro, Precio: 404250
Producto: Apple Tv 4k (wifi) A2737 De Voz 3.ª Generación 2022 4k 64gb Negro, Precio: 477099


Completo lista con la informacion de los items

In [77]:
items_data = [] # Lista con la informacion de cada item

# Obtener detalles de cada ítem
for item_id in all_item_ids:
    item = get_data(base_url_items, item_id)
    items_data.append({
        "id": item["id"],
        "producto": item["title"],
        "precio": item["price"],
        "site": item["site_id"],
        "seller_id": item["seller_id"]
    })

    time.sleep(0.2)  # Pequeña pausa para no saturar la API

# Ver los primeros 5 ítems obtenidos
print(items_data[:5])

[{'id': 'MLA1473995491', 'producto': 'Apple Tv 4k (wifi + Ethernet) A2843 De Voz 3.ª Generación 2022 4k 128gb Negro - Distribuidor Autorizado', 'precio': 499999, 'site': 'MLA', 'seller_id': 195832229}, {'id': 'MLA1449403463', 'producto': 'Apple Tv Hd A1625 De Voz 4.ª Generación 2015 Full Hd 32gb Negro Con 2gb De Memoria Ram', 'precio': 599990, 'site': 'MLA', 'seller_id': 119914491}, {'id': 'MLA1366426083', 'producto': 'Apple Tv 4k (wifi) A2737 De Voz 3.ª Generación 2022 4k 64gb Negro - Distribuidor Autorizado', 'precio': 405399, 'site': 'MLA', 'seller_id': 553476757}, {'id': 'MLA1555284784', 'producto': 'Apple Tv 4k (wifi + Ethernet) A2843 De Voz 3.ª Generación 2022 4k 128gb Negro', 'precio': 404250, 'site': 'MLA', 'seller_id': 1340767106}, {'id': 'MLA1399215546', 'producto': 'Apple Tv 4k (wifi) A2737 De Voz 3.ª Generación 2022 4k 64gb Negro', 'precio': 477099, 'site': 'MLA', 'seller_id': 208137579}]


### Guardamos CSV para item seleccionado

In [78]:
import pandas as pd

# Convertir a DataFrame
df = pd.DataFrame(items_data)

# Guardar en CSV

nombre_archivo = "meli_items_" + str(param_item) + ".csv" # el path del archivo contiende el nombre del item filtrado previamente
df.to_csv(nombre_archivo, index=False, sep=";")

print("Archivo CSV generado exitosamente.")

Archivo CSV generado exitosamente.
